In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
spark

#### 1. Read the dataset

In [266]:
churn_data = spark.read.csv(
    'file:///home/hadoop/Downloads/Telco_Customer_Churn.csv', 
    header=True,
    inferSchema=True)

In [267]:
churn_data.head()

Row(customerID='7590-VHVEG', gender='Female', SeniorCitizen=0, Partner='Yes', Dependents='No', tenure=1, PhoneService='No', MultipleLines='No phone service', InternetService='DSL', OnlineSecurity='No', OnlineBackup='Yes', DeviceProtection='No', TechSupport='No', StreamingTV='No', StreamingMovies='No', Contract='Month-to-month', PaperlessBilling='Yes', PaymentMethod='Electronic check', MonthlyCharges=29.85, TotalCharges='29.85', Churn='No')

#### 2. Data Exploration

a) how many customers records are in dataset

In [268]:
churn_data.count()

7043

In [269]:
len(churn_data.columns)

21

In [270]:
churn_data.columns

['customerID',
 'gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn']

b) what is distribution of gender among customers?

In [271]:
churn_data.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



In [272]:
churn_data.groupBy('gender').count().show()

+------+-----+
|gender|count|
+------+-----+
|Female| 3488|
|  Male| 3555|
+------+-----+



c) what is the distribution of contract type among customers?

In [273]:
churn_data.show()

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|  

In [274]:
churn_data.groupBy(['Contract']).count().show()

+--------------+-----+
|      Contract|count|
+--------------+-----+
|Month-to-month| 3875|
|      One year| 1473|
|      Two year| 1695|
+--------------+-----+



d) What is the percentage of customers who go churned

In [275]:
churn_data.select(['churn']).where("churn = 'Yes'").count() / churn_data.count() * 100

26.536987079369588

#### 3. Data Preprocessing
- check for missing values and handle missing data.

In [276]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [277]:
churn_data.select([count(when(isnull(col), col)).alias(col) for col in churn_data.columns]).show()

+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|         0|     0|            0|      0|         0|     0|           0|            0|              0|             0|           0|               0|          0|          0|              0|       0|               0| 

In [278]:
churn_data = churn_data.withColumn('TotalCharges', when(col('TotalCharges') == " ", None)\
                      .otherwise(col('TotalCharges')))

In [279]:
churn_data.select([count(when(isnull(col), col)).alias(col) for col in churn_data.columns]).show()

+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|         0|     0|            0|      0|         0|     0|           0|            0|              0|             0|           0|               0|          0|          0|              0|       0|               0| 

In [280]:
churn_data1 = churn_data.na.drop()

In [281]:
from pyspark.sql.types import FloatType

In [282]:
churn_data1 = churn_data.withColumn('TotalCharges', col('TotalCharges').cast(FloatType()))

In [283]:
churn_data1 = churn_data1.drop(col('customerID'))
churn_data = churn_data.drop(col('customerID'))

#### 4. Import Mllib

f) Convert categorical varibles into numerical format using one hot encoding or labe encoding

In [284]:
!pip3 install numpy pandas matplotlib seaborn

Defaulting to user installation because normal site-packages is not writeable


In [285]:
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml import Pipeline

convert categorical data to numeric
- StringIndexer: 
    - label categorical columns
    - string to numerical
- oneHotEncoder:
    - for each row, only one value will be true, rest all are false
    - for each category a column will be created, and for row with the category, the value will be set to 1, others to 0

In [286]:
churn_data.dtypes

[('gender', 'string'),
 ('SeniorCitizen', 'int'),
 ('Partner', 'string'),
 ('Dependents', 'string'),
 ('tenure', 'int'),
 ('PhoneService', 'string'),
 ('MultipleLines', 'string'),
 ('InternetService', 'string'),
 ('OnlineSecurity', 'string'),
 ('OnlineBackup', 'string'),
 ('DeviceProtection', 'string'),
 ('TechSupport', 'string'),
 ('StreamingTV', 'string'),
 ('StreamingMovies', 'string'),
 ('Contract', 'string'),
 ('PaperlessBilling', 'string'),
 ('PaymentMethod', 'string'),
 ('MonthlyCharges', 'double'),
 ('TotalCharges', 'string'),
 ('Churn', 'string')]

In [287]:
[item[0] for item in churn_data1.dtypes if item[1] == 'string']

['gender',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'Churn']

In [288]:
categorical_cols = [i.name for i in churn_data.schema.fields if isinstance(i.dataType, StringType)]

In [289]:
print(categorical_cols)

['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'TotalCharges', 'Churn']


In [290]:
print(categorical_cols[:-1])

['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'TotalCharges']


In [306]:
stages = []

for catcols in categorical_cols:
    stringindexer = StringIndexer(
        inputCol=catcols, 
        outputCol=catcols + 'Index')
    onehotencoder = OneHotEncoderEstimator(
        inputCols=[stringindexer.getOutputCol()], 
        outputCols=[catcols + 'classVec'])
    stages += [stringindexer, onehotencoder]

In [307]:
# do the same for numerical columns
numerical_cols = ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges']

In [308]:
assemblerInputs = [c + 'classVec' for c in categorical_cols[:-1]] + numerical_cols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol='features')
stages += [assembler]

In [309]:
label_stringIdx = StringIndexer(inputCol='Churn', outputCol='label')

In [310]:
stages += [label_stringIdx]

In [311]:
pipeline = Pipeline(stages=stages)

In [312]:
preprocessing = pipeline.fit(churn_data1)

In [313]:
churn_df = preprocessing.transform(churn_data1)

In [314]:
churn_df.show()

+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+-----------+--------------+------------+---------------+---------------+------------------+-----------------+--------------------+------------------+---------------------+--------------------+-----------------------+-------------------+----------------------+-----------------+--------------------+---------------------+------------------------+----------------+-------------------+----------------+-------------------+--------------------+-----------------------+-------------+----------------+---------------------+------------------------+------------------+---------------------+-----------------+--------------------+----------+-------------+--------------------+-----+
|gender|SeniorCitizen|Pa

In [315]:
churn_df.select(['Contract', 'ContractIndex', 'ContractclassVec']).show()

+--------------+-------------+----------------+
|      Contract|ContractIndex|ContractclassVec|
+--------------+-------------+----------------+
|Month-to-month|          0.0|   (2,[0],[1.0])|
|      One year|          2.0|       (2,[],[])|
|Month-to-month|          0.0|   (2,[0],[1.0])|
|      One year|          2.0|       (2,[],[])|
|Month-to-month|          0.0|   (2,[0],[1.0])|
|Month-to-month|          0.0|   (2,[0],[1.0])|
|Month-to-month|          0.0|   (2,[0],[1.0])|
|Month-to-month|          0.0|   (2,[0],[1.0])|
|Month-to-month|          0.0|   (2,[0],[1.0])|
|      One year|          2.0|       (2,[],[])|
|Month-to-month|          0.0|   (2,[0],[1.0])|
|      Two year|          1.0|   (2,[1],[1.0])|
|      One year|          2.0|       (2,[],[])|
|Month-to-month|          0.0|   (2,[0],[1.0])|
|Month-to-month|          0.0|   (2,[0],[1.0])|
|      Two year|          1.0|   (2,[1],[1.0])|
|      One year|          2.0|       (2,[],[])|
|      Two year|          1.0|   (2,[1],

In [316]:
churn_df1 = churn_df.select(['features', 'label'])
churn_df1.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                                         |label|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|(6559,[2,7,8,11,12,14,16,18,20,22,23,674,6556,6557,6558],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,29.85,29.850000381469727])                         |0.0  |
|(6559,[0,1,2,3,4,7,9,10,13,14,16,18,24,3345,6556,6557,6558],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,34.0,56.95,1889.5])                         |0.0  |
|(6559,[0,1,2,3,4,7,9,11,12,14,16,18,20,22,24,2250,6556,6557,6558],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,53.85,108.1500

#### split dataset into Train and Test

In [317]:
train, test = churn_df1.randomSplit([0.8, 0.2], seed = 2)

In [318]:
train.select(['features', 'label'])

DataFrame[features: vector, label: double]

In [319]:
#### Build Decision Tree ML Model

In [320]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier

tree = DecisionTreeClassifier(featuresCol='features', labelCol='label')
decision_model = tree.fit(train)

Py4JJavaError: An error occurred while calling o19808.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 489.0 failed 1 times, most recent failure: Lost task 0.0 in stage 489.0 (TID 1444, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$9: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.sort_addToSorter_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.ha

In [160]:
## Evaluate the model

In [187]:
predictions = decision_model.transform(test)

NameError: name 'decision_model' is not defined

In [ ]:
predictions.select([])